In [ ]:
import time
import pickle
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import xgboost as xgb
import optuna

from category_encoders import OrdinalEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, root_mean_squared_error
from optuna_integration import XGBoostPruningCallback

RANDOM_SEED = 42
COLORS = sns.color_palette()

np.random.seed(RANDOM_SEED)
pd.options.mode.copy_on_write = True
plt.style.use('fivethirtyeight')

print(f"Pandas version: {pd.__version__}")
print(f"NumPy version: {np.__version__}")
print(f"Seaborn version: {sns.__version__}")
print(f"XGBoost version: {xgb.__version__}")
print(f"CuPy version: {cp.__version__}")
print(f"Optuna version: {optuna.__version__}")


Getting Data

In [ ]:
bills = pd.read_csv('./bills.csv', usecols=['business_date', 'sales_revenue_with_tax', 'venue_xref_id'])

bills

In [ ]:
venues = pd.read_csv('./venues.csv')
venues.drop(columns=['start_of_day_offset'], inplace=True)

venues

In [ ]:
# merge grouped_bills and venues along the venue_xref_id axis
df = pd.merge(left=bills.groupby(['business_date', 'venue_xref_id'])['sales_revenue_with_tax'].agg(['sum', 'count']).reset_index(),
              right=venues,
              how='outer',
              on='venue_xref_id',
              validate='m:1')
df['business_date'] = pd.to_datetime(df['business_date'])
df.rename(columns={'sum': 'sales_revenue_with_tax', 'count': 'num_bills'}, inplace=True)

df

Data Visualization

In [ ]:
df['num_bills'].describe()

In [ ]:
g = sns.displot(data=df['num_bills'].loc[df['num_bills'] <= 500], 
                kde=True, height=5, aspect=2)
g.set_xlabels('Number of Daily Venue Bills')
g.set_titles('Distribution of Daily Venue Bills')
plt.show()

In [ ]:
g = sns.displot(data=df['num_bills'].loc[df['num_bills'] > 500], 
                kde=True, height=5, aspect=2)
g.set_xlabels('Number of Daily Venue Bills')
g.set_titles('Distribution of Daily Venue Bills')
plt.show()

Data Preprocessing

In [ ]:
def create_features(df: pd.DataFrame) -> pd.DataFrame:
    """
    Create new features for the data.
    """
    df = df.copy()

    # Add new features
    df['day_of_week'] = df['business_date'].dt.day_of_week
    df['day_of_year'] = df['business_date'].dt.day_of_year
    df['month'] = df['business_date'].dt.month
    df['quarter'] = df['business_date'].dt.quarter

    return df

def encode_categorical(df: pd.DataFrame, encoder: OrdinalEncoder, features: list) -> pd.DataFrame:
    df = df.copy()

    # Encode categorical data
    df = encoder.transform(df)

    # Specify categorical data type for XGBoost
    df[features] = df[features].astype('category')

    return df


In [ ]:
FEATURES = ['venue_xref_id', 'concept', 'city', 'country', 
            'day_of_week', 'day_of_year', 'month', 'quarter']
TARGET = 'sales_revenue_with_tax'

df = create_features(df)

ordinal_encoder = OrdinalEncoder(cols=FEATURES[:4])
ordinal_encoder.fit(df[FEATURES])

df[FEATURES] = encode_categorical(df[FEATURES], ordinal_encoder, FEATURES)
df

In [ ]:
# Save encoder for loading the encoder later and transforming new data
with open('encoder.obj', 'wb') as f:
    pickle.dump(ordinal_encoder, f)

Train Test Split

In [ ]:
train, test = train_test_split(df, test_size=0.2, shuffle=True, random_state=RANDOM_SEED)
test, valid = train_test_split(test, test_size=0.5, shuffle=True, random_state=RANDOM_SEED)
train.shape, valid.shape, test.shape

In [ ]:
X_train = train[FEATURES]
y_train = train[TARGET]

X_valid = valid[FEATURES]
y_valid = valid[TARGET]

X_test = test[FEATURES]
y_test = test[TARGET]

X_train.shape, y_train.shape, X_valid.shape, y_valid.shape, X_test.shape, y_test.shape

Tuning Tree Parameters

In [ ]:
def weighted_absolute_percentage_error(y_true, y_pred):
    return np.abs(y_true - y_pred).sum() / np.abs(y_true).sum()

In [ ]:
metric = weighted_absolute_percentage_error

base_params = {
    'objective': 'reg:squarederror',
    'eval_metric': metric,
    'learning_rate': 0.3,
    'enable_categorical': True,
    'booster': 'gbtree',
    'random_state': RANDOM_SEED,
    'device': 'cuda',
}

In [ ]:
# Define objective function for Optuna
def objective_fn(trial: optuna.trial.Trial) -> float:
    params = {
        'tree_method': trial.suggest_categorical('tree_method', ['approx', 'hist']),
        'max_depth': trial.suggest_int('max_depth', 3, 12),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 250),
        'subsample': trial.suggest_float('subsample', 0.1, 1.0),
        'colsample_bynode': trial.suggest_float('colsample_bynode', 0.1, 1.0),
        'reg_lambda': trial.suggest_float('reg_lambda', 0.001, 25, log=True),
    }

    params.update(base_params)
    pruning_callback = XGBoostPruningCallback(trial, f'validation_1-{metric.__name__}')

    regressor = xgb.XGBRegressor(n_estimators=10000,
                                 early_stopping_rounds=50,
                                 callbacks=[pruning_callback], 
                                 **params)
    
    regressor.fit(X_train, y_train,
                  eval_set=[(X_train, y_train), (X_valid, y_valid)],
                  verbose=False)

    trial.set_user_attr('best_iteration', regressor.best_iteration)
    return regressor.best_score


In [ ]:
sampler = optuna.samplers.TPESampler(seed=RANDOM_SEED)
study = optuna.create_study(direction='minimize', sampler=sampler)

tic = time.time()
while time.time() - tic < 300:
    study.optimize(objective_fn, n_trials=1)

In [ ]:
print('Stage 1 ==============================')
print(f'best score = {study.best_trial.value:.2%}')
print('boosting params ---------------------------')
print(f'fixed learning rate: {base_params['learning_rate']}')
print(f'best boosting round: {study.best_trial.user_attrs['best_iteration']}')
print('best tree params --------------------------')
for parameter, value in study.best_trial.params.items():
    print(parameter, ':', value)

Tuning Booster Parameters

In [ ]:
params = {}
params.update(base_params)
params.update(study.best_trial.params)
params['learning_rate'] = 0.01

In [ ]:
regressor = xgb.XGBRegressor(n_estimators=10000, early_stopping_rounds=50, **params)

regressor.fit(X_train, y_train,
              eval_set=[(X_train, y_train), (X_valid, y_valid)],
              verbose=False)

y_pred = regressor.predict(X_valid)

In [ ]:
print('Stage 2 ==============================')
print(f'best score = {weighted_absolute_percentage_error(y_valid, y_pred):.2%}')
print('boosting params ---------------------------')
print(f'fixed learning rate: {params['learning_rate']}')
print(f'best boosting round: {regressor.best_iteration}')

Model Training

In [ ]:
regressor = xgb.XGBRegressor(n_estimators=regressor.best_iteration, **params)
        
regressor.fit(X_train, y_train,
                    eval_set=[(X_train, y_train), (X_test, y_test)],
                    verbose=2)

Visualize Feature Importance

In [ ]:
feature_importance = pd.DataFrame(data=regressor.feature_importances_,
                                  index=regressor.feature_names_in_,
                                  columns=['importance'])
feature_importance.sort_values('importance').plot.barh(title='Feature Importance', legend=False)
plt.show()

Visualize Loss Curve

In [ ]:
loss_result = regressor.evals_result()

rmse_train_loss = loss_result['validation_0']['rmse']
rmse_test_loss = loss_result['validation_1']['rmse']

wape_train_loss = loss_result['validation_0']['weighted_absolute_percentage_error']
wape_test_loss = loss_result['validation_1']['weighted_absolute_percentage_error']

estimators = range(len(rmse_train_loss))

# fig, ax1 = plt.subplots(figsize=(16, 9))
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 9))

ax1.plot(estimators, rmse_train_loss, label='Train Loss')
ax1.plot(estimators, rmse_test_loss, label='Test Loss')
ax1.set_title('RMSE Loss')
ax1.legend(loc='best')

ax2.plot(estimators, wape_train_loss, label='Train Loss')
ax2.plot(estimators, wape_test_loss, label='Test Loss')
ax2.set_title('WAPE Loss')
ax2.legend(loc='best')

fig.supxlabel('Estimators')
fig.supylabel('Loss')

plt.show()


Forecasting

In [ ]:
y_pred = regressor.predict(X_test)
y_pred

In [ ]:
pred_df = pd.DataFrame({'business_date': test['business_date'],
                        'venue_xref_id': test['venue_xref_id'],
                        'target': y_test, 
                        'prediction': y_pred})
pred_df.head()

In [ ]:
# fig, ax1 = plt.subplots(figsize=(15, 10))
# ax1.set_xlabel('Business Date')
# ax1.set_ylabel('Sales Revenue')
# ax1.scatter(pred_df['business_date'], pred_df['target'], color=COLORS[0])

# ax2 = ax1.twinx()
# ax2.scatter(pred_df['business_date'], pred_df['prediction'], color=COLORS[3])
# ax2.set_axis_off()

# fig.suptitle('Target vs Prediction by Date')
# fig.legend(['target', 'prediction'])
# plt.show()

In [ ]:
# fig, ax1 = plt.subplots(figsize=(15, 10))
# ax1.set_xlabel('Encoded Venue ID')
# ax1.set_ylabel('Sales Revenue')
# ax1.scatter(pred_df['venue_xref_id'], pred_df['target'], color=COLORS[0])

# ax2 = ax1.twinx()
# ax2.scatter(pred_df['venue_xref_id'], pred_df['prediction'], color=COLORS[3])
# ax2.set_axis_off()

# fig.suptitle('Target vs Prediction by Venue')
# fig.legend(['target', 'prediction'])
# plt.show()

Score

In [ ]:
train_pred = regressor.predict(X_train)

In [ ]:
wape_train_score = weighted_absolute_percentage_error(y_train, train_pred)
print(f'WAPE Score on Train set: {wape_train_score:.2%}')

wape_test_score = weighted_absolute_percentage_error(pred_df['target'], pred_df['prediction'])
print(f'WAPE Score on Test set: {wape_test_score:.2%}')

In [ ]:
rmse_train_score = root_mean_squared_error(y_train, train_pred)
print(f'RMSE Score on Train set: {rmse_train_score:.2f}')

rmse_test_score = root_mean_squared_error(pred_df['target'], pred_df['prediction'])
print(f'RMSE Score on Test set: {rmse_test_score:.2f}')

In [ ]:
mae_train_score = mean_absolute_error(y_train, train_pred)
print(f'MAE Score on Train set: {mae_train_score:.2f}')

mae_test_score = mean_absolute_error(pred_df['target'], pred_df['prediction'])
print(f'MAE Score on Test set: {mae_test_score:.2f}')

Calculate Error

In [ ]:
pred_df['error'] = np.abs(pred_df['target'] - pred_df['prediction'])

print(f"Best 10 Predictions:\n{pred_df.sort_values('error', ascending=True).head(10)}")

In [ ]:
print(f"Worst 10 Predictions:\n{pred_df.sort_values('error', ascending=False).head(10)}")

Retrain on All Data

In [ ]:
X_all = df[FEATURES]
y_all = df[TARGET]

X_all.shape, y_all.shape

In [ ]:
final_model = xgb.XGBRegressor(n_estimators=regressor.best_iteration, **params)
        
final_model.fit(X_all, y_all,
                eval_set=[(X_all, y_all)],
                verbose=100)

Save Model

In [ ]:
final_model.save_model('CxC_TouchBistro_Forecaster.json')